In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../plot-vbd-floor/')

import numpy as np
import matplotlib.pyplot as plt
import scipy

from plot_helper import *
from constants import STANDING_DATA, STANDING_DIS_DATA, STANDING_DIS_STD, BEACONS_LOC
from dataclasses import dataclass
STANDING_DIS_DATA.shape, BEACONS_LOC.shape

((48, 6), (6, 2))

## Genarate Prior samples
Particle filter is implemented in 2D space.

In [72]:
from my_particle_filter import *

robot = RobotState(Point(6.5, 13.5),angle=0)
N = 200
particles = [RobotState(noise=Noise(0.1, 5/180*np.pi, 3)) for _ in range(N)]
robot.angle

0.0

## Update prior with infomation from measurements

In [ ]:
for step in range(STANDING_DIS_DATA.shape[0]):
    # robot.move(.0, 0.)
    # meas = robot.sense()
    meas = STANDING_DIS_DATA[step,:]
    hdg = STANDING_DATA[step,-1]

    # plot pre particles pre measurement
    fig, ax = plt.subplots(figsize=(8,10))
    ax = plot_radiomap(ax)
    ax = visualization(ax, robot=robot, step=step, particles=particles,particles_resampled=None)
    fig.savefig(f'./img/PF{step}.jpg')
    ax.plot()

    for p in particles:
        p.move(hdg/180*np.pi, 0.0)
    #
    weights = [p.meas_probability(meas) for p in particles]
    weights = np.array(weights) + 1.e-300
    weights = weights/weights.sum()
    # resampled
    particles_resampled = []
    index = int(random.random() * N)
    beta = 0.
    for _ in range(N):
        beta += random.random() * 2. * max(weights)
        while beta > weights[index]:
            beta -= weights[index]
            index = (index + 1) % N
        particles_resampled.append(copy(particles[index]))
    print('Num of particles: ', len(particles))
    print('Num of particles_resampled: ', len(particles_resampled))
    # fig, ax = plt.subplots(figsize=(8,10))
    # ax = plot_radiomap(ax)
    ax = visualization(ax, robot=robot, step=step, particles=particles,particles_resampled=particles_resampled)
    fig.savefig(f'./img/PF{step}_meas.jpg')
    ax.plot()
    # resample
    particles = particles_resampled

In [74]:
indexes = systematic_resample(weights=weights)

In [75]:


particles_np, weights = resample_from_index(particles=particles, weights=weights, indexes=indexes)

In [76]:
particles_np

array([[ 5.66749876, 13.92065979],
       [ 5.66749876, 13.92065979],
       [ 5.64666873, 13.88269084],
       [ 5.94786948, 14.03389334],
       [ 5.56756645, 13.96385459],
       [ 5.51488637, 13.95268234],
       [ 5.87923595, 14.12844673],
       [ 5.87923595, 14.12844673],
       [ 5.87923595, 14.12844673],
       [ 6.17892066, 14.21591558],
       [ 6.58982621, 14.36759458],
       [ 6.14225767, 14.23495533],
       [ 6.06768259, 14.17611636],
       [ 5.7668833 , 13.93801143],
       [ 5.7668833 , 13.93801143],
       [ 5.7668833 , 13.93801143],
       [ 5.58208801, 13.8498463 ],
       [ 5.78431917, 13.9756018 ],
       [ 5.54581593, 13.87384078],
       [ 5.54581593, 13.87384078],
       [ 5.54581593, 13.87384078],
       [ 5.69892383, 13.96920661],
       [ 5.69892383, 13.96920661],
       [ 5.64498933, 13.95416289],
       [ 5.58038225, 13.9317199 ],
       [ 5.6064576 , 13.94251978],
       [ 5.86512767, 14.05371248],
       [ 5.98931413, 14.1075206 ],
       [ 5.9492177 ,